In [1]:
'''
Notes for this version:
1. wrote the whole thing into two functions
2. used Ying's method for reading into drug data, 
        Ying and Joshua's method of merging data (using pd.merge function), 
        Ying's method for extra credit (modified)
        (Applause!!)
3. a few minor modification on the details of the code (more informative variable names, more concise, etc.)
4. for discussion: 
1) filtered before merge, think this will make it faster
2) for extra credit, separated entries with multiple genes and drugs 
3) other things to improve?

# Result files are uploaded onto repo, added 'V1' before original file names
'''


"\nNotes for this version:\n1. wrote the whole thing into two functions\n2. used Ying's method for reading into drug data, \n        Ying and Joshua's method of merging data (using pd.merge function), \n        Ying's method for extra credit (modified)\n        (Applause!!)\n3. a few minor modification on the details of the code (more informative variable names, more concise, etc.)\n4. for discussion: \n1) filtered before merge, think this will make it faster\n2) for extra credit, separated entries with multiple genes and drugs \n3) other things to improve?\n\n# Result files are uploaded onto repo, added 'V1' before original file names\n"

In [2]:
import pandas as pd

In [3]:
def merge_23andme_drug(me23_file = '~/Desktop/575FinalProject/23andme_v5_hg19_ref.txt', drug_file = '~/Desktop/575FinalProject/var_drug_ann.tsv'):
    '''This function takes file paths for the 23andMe and var_drug_ann files to then merge these files 
       into a single, cleaned data frame. The default paths for these files are based on the GitHub repo
       which can be cloned at:https://github.com/tmstew/Final_Project_575.git.
       
       Note: This method currently ignores 13 lines of the file that have additional information/are not able to be read in properly. 
             We are ignoring these lines base on direstion from Christina.
       '''
    data23me_names = ['Chromosomes', 'Position', 'Variant', 'Allele']
    data23me = pd.read_csv(me23_file, sep = '\t', names = data23me_names)
    data_var_drug = pd.read_csv(drug_file, sep = '\t', error_bad_lines = False)
    data_var_drug_filtered = data_var_drug.loc[(data_var_drug['Significance'] == 'yes') & (data_var_drug['Phenotype Category'] == 'efficacy')]
    me23_drug_merged = pd.merge(data23me, data_var_drug_filtered).reset_index(drop = True)
    output_df = me23_drug_merged.loc[: ,['Variant', 'Gene', 'Chemical', 'Notes', 'Sentence', 'Alleles', 'Allele']]
    output_df.columns = ['dbSNP_ID', 'GENE_SYMBOL', 'DRUG_NAME', 'NOTES', 'SENTENCE', 'ALLELE_PharmGKB', 'ALLELE_23andme']
    output_df_cleaned = output_df.fillna('(no entry)').replace(to_replace = '', value = '(no entry)')
    return output_df_cleaned

In [5]:
# Call the merge_23andme_drug function 
final_df = merge_23andme_drug()
final_df.to_csv('23andme_PharmGKB_map.tsv', sep = '\t')

FileNotFoundError: [Errno 2] File b'C:\\Users\\tstew/Desktop/575FinalProject/23andme_v5_hg19_ref.txt' does not exist: b'C:\\Users\\tstew/Desktop/575FinalProject/23andme_v5_hg19_ref.txt'

In [94]:
def gene_drug_group(merged_df):
    '''This function take in the combined, cleaned data frame from the merge_23andme_drug function.
       This function will then group the data by dbSNP_ID and GENE_SYMBOL/DRUG_NAME, outputting these
       in a new dataframe.'''
    small_df = merged_df.loc[:,['dbSNP_ID','GENE_SYMBOL','DRUG_NAME']]
    new_df = small_df
    for name_string in small_df.columns:
        new_series = small_df[name_string].str.split(',', expand = True).stack()
        new_series.index = new_series.index.droplevel(-1)
        new_series.name = name_string
        new_df = new_df.drop(axis = 1, labels = name_string).join(new_series)
    group = new_df.groupby(['GENE_SYMBOL','DRUG_NAME'])['dbSNP_ID'].apply(';'.join)
    df_group = pd.DataFrame(group)
    return df_group

In [97]:
# Call the gene_drug_group function
df_group = gene_drug_group(final_df)
df_group.to_csv('extra_credit.tsv', sep="\t")